In [0]:
spark.conf.set(
  "fs.azure.account.key.snowflakesession.blob.core.windows.net",
  "QFOWMa+1PeHMd8oTgZqPeddxMAzLv1qlIO0qdoeUN2lg1qYlMHlfDQUCvLju+ic/i9RnJSyadnch+ASts7qAeQ=="
)
df_raw = (
    spark.read
         .option("multiLine", "true")
         .json("wasbs://adf@snowflakesession.blob.core.windows.net/insurance_claims.json")
)

df_raw.printSchema()
df_raw.show(truncate=False)



root
 |-- claim_date: string (nullable = true)
 |-- claim_id: string (nullable = true)
 |-- claimant: struct (nullable = true)
 |    |-- customer_id: string (nullable = true)
 |    |-- region: string (nullable = true)
 |-- hospital: struct (nullable = true)
 |    |-- city: string (nullable = true)
 |    |-- name: string (nullable = true)
 |-- payment: struct (nullable = true)
 |    |-- payment_mode: string (nullable = true)
 |    |-- status: string (nullable = true)
 |-- policy: struct (nullable = true)
 |    |-- coverage_amount: long (nullable = true)
 |    |-- insurance_type: string (nullable = true)
 |    |-- policy_number: string (nullable = true)
 |-- treatment: struct (nullable = true)
 |    |-- claim_amount: long (nullable = true)
 |    |-- diagnosis: string (nullable = true)

+----------+--------+-------------------+--------------------------+----------------+-------------------------+-----------------+
|claim_date|claim_id|claimant           |hospital                  |payment

In [0]:
from pyspark.sql.functions import col, to_date, struct

df_standardized = (
    df_raw
    .select(
        col("claim_id"),
        col("policy.policy_number").alias("policy_number"),
        col("policy.insurance_type").alias("insurance_type"),
        col("policy.coverage_amount").alias("coverage_amount"),
        col("claimant.customer_id").alias("customer_id"),
        col("claimant.region").alias("region"),
        col("treatment.claim_amount").alias("claim_amount"),
        col("payment.status").alias("payment_status"),
        col("payment.payment_mode").alias("payment_mode"),
        to_date(col("claim_date")).alias("claim_date"),

        # Correct way to keep full JSON as semi-structured data
        struct("*").alias("raw_json")
    )
)


In [0]:
sfOptions = {
    "sfURL": "FJPKTEI-SZ94461.snowflakecomputing.com",
    "sfUser": "ICICI_USER",
    "sfPassword": "Icici@123",
    "sfWarehouse": "ICICI_WH",
    "sfDatabase": "ICICI_INSURANCE_DB",
    "sfSchema": "CLAIMS_SCHEMA"
}

In [0]:
df_standardized.count()
df_standardized.select("payment_status").distinct().show()


+--------------+
|payment_status|
+--------------+
|      APPROVED|
|          NULL|
+--------------+

